In [7]:
import pandas as pd
import numpy as np

from os.path import exists
from os import makedirs

dir = './generated_data/'
if not exists(dir):
    makedirs(dir)

---

# DATA PROCESS

---

**DATASET 1:  "Población de 16 años y más por titulación académica y lugar de
nacimiento" (1997-2025)**

We used the first dataset to obtain information about the educational level of the population in the different neighborhoods and districts of Barcelona.

In [50]:
# We put all datasets on a list
all_files=[]
for year in range(1997,2026): # 1997-2025
    all_files.append(f'../data/education_level/{year}_education_level.csv')

# We read all datasets and prepare them
df_list=[]
for file in all_files:

    df=pd.read_csv(file, sep=',')

    # We delete the '..' from the column 'Value'
    df['Value']=pd.to_numeric(df['Value'], errors='coerce') # We change '..' to NaN
    df=df.dropna(subset=['Value']) # We delete the NaN
    
    df_list.append(df)

# We combine all dataframes in one
education_df=pd.concat(df_list, ignore_index=True)

# We save the clean dataset
education_df.to_csv('generated_data/education_1997-2025.csv', index=False, encoding='utf-8')
education_df

,Year_Reference,District,Neighborhood,Value,Education_Level,Birth_Place
0,1997,Ciutat Vella,el Raval,2369.0,Less than primary education,City of Barcelona
1,1997,Ciutat Vella,el Raval,601.0,Less than primary education,Rest of Catalonia
2,1997,Ciutat Vella,el Raval,4241.0,Less than primary education,Rest of Spain
3,1997,Ciutat Vella,el Raval,57.0,Less than primary education,Rest of European Union
4,1997,Ciutat Vella,el Raval,540.0,Less than primary education,Rest of World
...,...,...,...,...,...,...
59760,2025,Sant Martí,la Verneda i la Pau,550.0,Tertiary education,Rest of Spain
59761,2025,Sant Martí,la Verneda i la Pau,269.0,Tertiary education,Rest of European Union
59762,2025,Sant Martí,la Verneda i la Pau,2222.0,Tertiary education,Rest of World
59763,2025,Sant Martí,la Verneda i la Pau,31.0,Not available,City of Barcelona


For the bubble chart plot we need to extract from this data frame:
- % of total people with postmandatory studies in each neighborhood.
- % of inmigrants with postmandatory studies in each neighborhood.

In [51]:
# We read the dataset
df=pd.read_csv('generated_data/education_1997-2025.csv')

# We are interested on postmandatory studies, so let's define what they are
postmandatory_studies=['Upper secondary or post-secondary non-tertiary education', 'Tertiary education']

# Let's define inmigrants (out of Spain)
inmigrant_types=['Rest of European Union', 'Rest of World']

# Let's obtain the total population in each neighborhood
total_pop=df.groupby(['Year_Reference', 'District', 'Neighborhood'])['Value'].sum().reset_index()
total_pop.columns=['Year_Reference', 'District', 'Neighborhood', 'Total_Population']

# Let's obtain how many of these people have postmandatory studies
postmandatory=df[df['Education_Level'].isin(postmandatory_studies)] # We find the studies
postmandatory_total=postmandatory.groupby(['Year_Reference', 'District', 'Neighborhood'])['Value'].sum().reset_index() # We group by neighborhoods
postmandatory_total.columns=['Year_Reference', 'District', 'Neighborhood', 'Total_Postmandatory']

# Let's obtain the total inmigrants in each neighborhood
inmigrants=df[df['Birth_Place'].isin(inmigrant_types)] # We find the inmigrants
total_inmigrants=inmigrants.groupby(['Year_Reference', 'District', 'Neighborhood'])['Value'].sum().reset_index() # We group by neighborhoods
total_inmigrants.columns = ['Year_Reference', 'District', 'Neighborhood', 'Total_Inmigrants']

# Now, how many inmigrants have postmandatory studies
postmandatory_inmigrants=inmigrants[inmigrants['Education_Level'].isin(postmandatory_studies)] # We find the studies
postmandatory_inmigrants=postmandatory_inmigrants.groupby(['Year_Reference', 'District', 'Neighborhood'])['Value'].sum().reset_index() # We group by neighborhoods
postmandatory_inmigrants.columns=['Year_Reference', 'District', 'Neighborhood', 'Postmandatory_Inmigrants']

# Combine everything in the same DataFrame
percentages=total_pop.merge(postmandatory_total, on=['Year_Reference', 'District', 'Neighborhood'])
percentages=percentages.merge(postmandatory_inmigrants, on=['Year_Reference', 'District', 'Neighborhood'], how='left')
percentages=percentages.merge(total_inmigrants, on=['Year_Reference', 'District', 'Neighborhood'], how='left')

# Obtain percentages
percentages['Percentage_Postmandatory']=(percentages['Total_Postmandatory']/percentages['Total_Population'])*100
percentages['Percentage_Postmandatory_Inmigrants']=(percentages['Postmandatory_Inmigrants']/percentages['Total_Inmigrants'])*100

# Clean NaN from percentages: Refill with zero, the NaN is caused because there are no inmigrants
percentages['Percentage_Postmandatory_Inmigrants']=percentages['Percentage_Postmandatory_Inmigrants'].fillna(0)

# We save the dataset 
percentages.to_csv('generated_data/postcompulsory_percentages_1997-2025.csv', index=False, encoding='utf-8')
percentages

,Year_Reference,District,Neighborhood,Total_Population,Total_Postmandatory,Postmandatory_Inmigrants,Total_Inmigrants,Percentage_Postmandatory,Percentage_Postmandatory_Inmigrants
0,1997,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",17854.0,5082.0,697.0,1728.0,28.464210,40.335648
1,1997,Ciutat Vella,el Barri Gòtic,12255.0,4054.0,554.0,1184.0,33.080375,46.790541
2,1997,Ciutat Vella,el Raval,30337.0,6067.0,1164.0,3542.0,19.998681,32.862789
3,1997,Ciutat Vella,la Barceloneta,13098.0,2089.0,184.0,543.0,15.949000,33.885820
4,1997,Eixample,Sant Antoni,32270.0,13088.0,728.0,1293.0,40.557794,56.303171
...,...,...,...,...,...,...,...,...,...
2112,2025,Sarrià-Sant Gervasi,Sant Gervasi - la Bonanova,22690.0,18493.0,4586.0,5722.0,81.502865,80.146802
2113,2025,Sarrià-Sant Gervasi,Sarrià,21032.0,16868.0,4107.0,5043.0,80.201598,81.439619
2114,2025,Sarrià-Sant Gervasi,"Vallvidrera, el Tibidabo i les Planes",4150.0,3010.0,862.0,1089.0,72.530120,79.155188
2115,2025,Sarrià-Sant Gervasi,el Putxet i el Farró,26503.0,21680.0,6522.0,7790.0,81.802060,83.722721


On the other hand, to analize the different levels of education per district we need to extract from the dataset the percentages of education per district.

In [52]:
# We read the data
df=pd.read_csv('generated_data/education_1997-2025.csv')

# We group the population by years, districts and level of education
group=df.groupby(['Year_Reference', 'District', 'Education_Level'])['Value'].sum().reset_index()

# We group now the population by years and districts (all levels of education). We have to include also the not available.
total_per_district=group.groupby(['Year_Reference', 'District'])['Value'].sum().reset_index()
total_per_district.rename(columns={'Value': 'Total_District'}, inplace=True)

# We group both dataframes and obtain percentages
df=pd.merge(group, total_per_district, on=['Year_Reference', 'District'])
df['Percentage']=(df['Value']/df['Total_District'])*100

df.to_csv('generated_data/education_percentages_1997-2025.csv', index=False, encoding='utf-8')
df

,Year_Reference,District,Education_Level,Value,Total_District,Percentage
0,1997,Ciutat Vella,Less than primary education,17478.0,73544.0,23.765365
1,1997,Ciutat Vella,Lower secondary education,11649.0,73544.0,15.839497
2,1997,Ciutat Vella,Not available,2049.0,73544.0,2.786087
3,1997,Ciutat Vella,Primary education,25076.0,73544.0,34.096595
4,1997,Ciutat Vella,Tertiary education,6476.0,73544.0,8.805613
...,...,...,...,...,...,...
1735,2025,Sarrià-Sant Gervasi,Lower secondary education,15231.0,130467.0,11.674216
1736,2025,Sarrià-Sant Gervasi,Not available,288.0,130467.0,0.220745
1737,2025,Sarrià-Sant Gervasi,Primary education,7760.0,130467.0,5.947864
1738,2025,Sarrià-Sant Gervasi,Tertiary education,72830.0,130467.0,55.822545


**DATASET 2: "Indicadores socioeconómicos. Valores. Por barrios de Barcelona" (2015-2022)**

This second dataset provides information about some socioeconomic indices in the different neighborhoods of Barcelona.

In [53]:
# We put all datasets on a list
all_files=[]
for year in range(2015,2023): # 2015-2022
    all_files.append(f'../data/indicadores_socioeconomicos/ist-14075-15023-mun_{year}.csv')

# We read all datasets and clean them
df_list=[]
for file in all_files:

    df=pd.read_csv(file, sep=';', decimal=',')

    # We delete the column 'estado' because it doesn't have values, and 'municipio' because we already know that we are in Barcelona
    df.drop(columns=['municipio','estado'], inplace=True)

    # We change column 'valor' to numeric
    df['valor']=pd.to_numeric(df['valor'].astype(str).str.replace(',', '.'), errors='coerce')

    # Let's change the indicators of 'concepto' into columns. This will agrupate the neighborhoods instead of having them repeat
    df=df.pivot_table(
        index=['año', 'barrios de Barcelona'], 
        columns='concepto', 
        values='valor'
    ).reset_index()   
    df.columns.name=None

    # We are going to delete the columns of 'población con estudios bajos' and 'población joven sin estudios postobligatorios', as this information
    # is incomplete for some years and we have it in other datasets 
    columns=['población con estudios bajos (%)', 'población joven sin estudios postobligatorios (%)'] # Columns we want to delete
    existing_columns=[] # Let's check if they exist, and delete them in that case
    for col in columns:
        if col in df.columns:          
            existing_columns.append(col)  
    if existing_columns:
        df.drop(columns=existing_columns, inplace=True)

    df_list.append(df)

# We combine all dataframes in one
indices_df=pd.concat(df_list, ignore_index=True)

# We save the clean dataset
indices_df.to_csv('generated_data/indices_2015-2022.csv', index=False, encoding='utf-8')
indices_df

,año,barrios de Barcelona,extranjeros de países de renta baja o media (%),población ocupada (%),renta media por persona (€),trabajadores de baja calificación (%)
0,2015,Baró de Viver,9.6,45.7,8036.0,22.9
1,2015,Can Baró,8.9,60.6,13555.0,9.2
2,2015,Can Peguera,5.6,46.0,9029.0,18.8
3,2015,Canyelles,3.4,59.3,12377.0,14.4
4,2015,Ciutat Meridiana,26.7,47.8,7687.0,27.7
...,...,...,...,...,...,...
587,2022,la Vila de Gràcia,9.6,68.2,19570.0,3.9
588,2022,les Corts,7.7,68.6,23381.0,3.7
589,2022,les Roquetes,21.1,60.5,10946.0,21.0
590,2022,les Tres Torres,4.3,66.9,33382.0,2.2


**DATASET 3: "Población de 16 años y más por titulación académica y continente de nacimiento" (1997-2025)**

This last dataset gives us information about the educational level of the population in each district, classifying them by the continent where they were born.

In [54]:
# We read the dataset

df=pd.read_csv('../data/nivel_educativo_continentes.csv', sep=',')
df

,Territorio,Tipo de territorio,Titulación académica,01 ene 1997,01 ene 1997.1,01 ene 1997.2,01 ene 1997.3,01 ene 1997.4,01 ene 1997.5,01 ene 1998,...,01 ene 2024.2,01 ene 2024.3,01 ene 2024.4,01 ene 2024.5,01 ene 2025,01 ene 2025.1,01 ene 2025.2,01 ene 2025.3,01 ene 2025.4,01 ene 2025.5
0,Territorio,Tipo de territorio,Titulación académica,África,América,Asia,Europa,Oceania,No consta,África,...,Asia,Europa,Oceania,No consta,África,América,Asia,Europa,Oceanía,No consta
1,Barcelona,Municipi,Sin estudios,1328,901,403,196617,-,-,1510,...,373,16532,-,-,300,178,429,14114,-,-
2,Barcelona,Municipi,"Estudios primarios, certificado de escolaridad...",2139,3148,1168,333835,-,14,2505,...,25914,153040,28,34,12331,25868,27068,152733,40,35
3,Barcelona,Municipi,"Bachillerato elemental, graduado escolar, ESO,...",1027,2510,531,235277,5,10,1105,...,28006,233800,62,39,12606,63612,31217,227241,68,43
4,Barcelona,Municipi,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",1411,6516,1419,268125,28,24,1565,...,16409,272394,166,84,8450,93999,18597,270442,180,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Sant Martí,Districte,"Estudios primarios, certificado de escolaridad...",265,349,66,50462,-,-,327,...,3925,26522,-,10,1926,3275,4080,26285,5,12
63,Sant Martí,Districte,"Bachillerato elemental, graduado escolar, ESO,...",169,314,33,36970,-,-,181,...,4044,40362,13,-,1873,8105,4480,39337,11,-
64,Sant Martí,Districte,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",150,698,83,33327,-,-,169,...,2185,40458,13,9,1169,12530,2552,40344,17,8
65,Sant Martí,Districte,"Estudios universitarios, CFGS grado superior",98,449,81,17508,-,5,113,...,3351,49775,90,38,1103,17044,3876,51122,105,37


In [55]:
# We delete the raws of all Barcelona because we are interested in the districts
df=df[df['Territorio']!='Barcelona'].copy()

# We also delete the column 'Tipo de territorio' because all of them are districts, 
df=df.drop('Tipo de territorio', axis=1)
df

,Territorio,Titulación académica,01 ene 1997,01 ene 1997.1,01 ene 1997.2,01 ene 1997.3,01 ene 1997.4,01 ene 1997.5,01 ene 1998,01 ene 1998.1,...,01 ene 2024.2,01 ene 2024.3,01 ene 2024.4,01 ene 2024.5,01 ene 2025,01 ene 2025.1,01 ene 2025.2,01 ene 2025.3,01 ene 2025.4,01 ene 2025.5
0,Territorio,Titulación académica,África,América,Asia,Europa,Oceania,No consta,África,América,...,Asia,Europa,Oceania,No consta,África,América,Asia,Europa,Oceanía,No consta
7,Ciutat Vella,Sin estudios,641,136,227,16474,-,-,762,156,...,99,687,-,-,60,6,104,560,-,-
8,Ciutat Vella,"Estudios primarios, certificado de escolaridad...",620,451,565,23440,-,-,734,566,...,8040,7452,5,-,2473,1458,8309,7343,7,-
9,Ciutat Vella,"Bachillerato elemental, graduado escolar, ESO,...",129,249,193,11077,-,-,144,278,...,8492,10340,15,6,2452,3835,9451,10278,11,6
10,Ciutat Vella,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",197,614,422,9578,5,-,233,733,...,3632,10774,51,8,1503,6694,4079,10787,52,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Sant Martí,"Estudios primarios, certificado de escolaridad...",265,349,66,50462,-,-,327,420,...,3925,26522,-,10,1926,3275,4080,26285,5,12
63,Sant Martí,"Bachillerato elemental, graduado escolar, ESO,...",169,314,33,36970,-,-,181,345,...,4044,40362,13,-,1873,8105,4480,39337,11,-
64,Sant Martí,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",150,698,83,33327,-,-,169,795,...,2185,40458,13,9,1169,12530,2552,40344,17,8
65,Sant Martí,"Estudios universitarios, CFGS grado superior",98,449,81,17508,-,5,113,518,...,3351,49775,90,38,1103,17044,3876,51122,105,37


In [56]:
# Wee will mantain the first two columns and modificate all the others
old_columns=['Territorio', 'Titulación académica']

columns_to_modificate=[]
for col in df.columns:
    if col not in old_columns:
        columns_to_modificate.append(col)

# We change the format from wide to long
df=df.melt(
    id_vars=old_columns, # We mantain these ones
    value_vars=columns_to_modificate, # Each column generates a new row for each original row
    var_name='Columna_Original', # Name of the new column where the original columns are 
    value_name='Poblacion' # Column with the values
)
df

,Territorio,Titulación académica,Columna_Original,Poblacion
0,Territorio,Titulación académica,01 ene 1997,África
1,Ciutat Vella,Sin estudios,01 ene 1997,641
2,Ciutat Vella,"Estudios primarios, certificado de escolaridad...",01 ene 1997,620
3,Ciutat Vella,"Bachillerato elemental, graduado escolar, ESO,...",01 ene 1997,129
4,Ciutat Vella,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",01 ene 1997,197
...,...,...,...,...
10609,Sant Martí,"Estudios primarios, certificado de escolaridad...",01 ene 2025.5,12
10610,Sant Martí,"Bachillerato elemental, graduado escolar, ESO,...",01 ene 2025.5,-
10611,Sant Martí,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",01 ene 2025.5,8
10612,Sant Martí,"Estudios universitarios, CFGS grado superior",01 ene 2025.5,37


In [57]:
# We clean the values
df['Poblacion']=df['Poblacion'].replace(['-', ''], np.nan)
df['Poblacion']=pd.to_numeric(df['Poblacion'], errors='coerce')

# We create an array with the values of Columna_Original (without repeating)
unique_columns=df['Columna_Original'].unique()

# Let's reconstruct the dataframe (we just reconstruct over the second column)
columns_df=pd.DataFrame({
        'Columna_Original': unique_columns,
        'Orden_Original': range(len(unique_columns))
    })

# Each date has 6 columns (one for each continent)
n_continents=6
columns_df['Grupo']=columns_df['Orden_Original']//n_continents # Each group is a date
columns_df['Posicion_Grupo']=columns_df['Orden_Original']%n_continents # Position in the group
map_continents={
        0: 'África', 1: 'América', 2: 'Asia', 
        3: 'Europa', 4: 'Oceanía', 5: 'No consta'
    }

# Extract continents
columns_df['Continente'] = columns_df['Posicion_Grupo'].map(map_continents)

# Extract dates
dates=columns_df.groupby('Grupo')['Columna_Original'].first().tolist()
columns_df['Fecha'] = columns_df['Grupo'].map(lambda x: dates[x])

# Let's add them to the original df
final_map=columns_df.set_index('Columna_Original')[['Fecha', 'Continente']].to_dict('index')
df['Fecha']=df['Columna_Original'].map(lambda x: final_map[x]['Fecha'])
df['Continente']=df['Columna_Original'].map(lambda x: final_map[x]['Continente'])

#Extract the year
df['Año']=df['Fecha'].str.extract(r'(\d{4})').astype(float)

# We clean data again
df=df.dropna(subset=['Poblacion'])
df

,Territorio,Titulación académica,Columna_Original,Poblacion,Fecha,Continente,Año
1,Ciutat Vella,Sin estudios,01 ene 1997,641.0,01 ene 1997,África,1997.0
2,Ciutat Vella,"Estudios primarios, certificado de escolaridad...",01 ene 1997,620.0,01 ene 1997,África,1997.0
3,Ciutat Vella,"Bachillerato elemental, graduado escolar, ESO,...",01 ene 1997,129.0,01 ene 1997,África,1997.0
4,Ciutat Vella,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",01 ene 1997,197.0,01 ene 1997,África,1997.0
5,Ciutat Vella,"Estudios universitarios, CFGS grado superior",01 ene 1997,73.0,01 ene 1997,África,1997.0
...,...,...,...,...,...,...,...
10605,Sant Andreu,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",01 ene 2025.5,10.0,01 ene 2025,No consta,2025.0
10606,Sant Andreu,"Estudios universitarios, CFGS grado superior",01 ene 2025.5,14.0,01 ene 2025,No consta,2025.0
10609,Sant Martí,"Estudios primarios, certificado de escolaridad...",01 ene 2025.5,12.0,01 ene 2025,No consta,2025.0
10611,Sant Martí,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",01 ene 2025.5,8.0,01 ene 2025,No consta,2025.0


In [58]:
# We can now delete Columna_Original and Fecha, as they are repeated information
df=df.drop(['Columna_Original','Fecha'], axis=1)

# We save our dataframe
df.to_csv('generated_data/education_continents_1997-2025.csv', index=False, encoding='utf-8')
df

,Territorio,Titulación académica,Poblacion,Continente,Año
1,Ciutat Vella,Sin estudios,641.0,África,1997.0
2,Ciutat Vella,"Estudios primarios, certificado de escolaridad...",620.0,África,1997.0
3,Ciutat Vella,"Bachillerato elemental, graduado escolar, ESO,...",129.0,África,1997.0
4,Ciutat Vella,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",197.0,África,1997.0
5,Ciutat Vella,"Estudios universitarios, CFGS grado superior",73.0,África,1997.0
...,...,...,...,...,...
10605,Sant Andreu,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",10.0,No consta,2025.0
10606,Sant Andreu,"Estudios universitarios, CFGS grado superior",14.0,No consta,2025.0
10609,Sant Martí,"Estudios primarios, certificado de escolaridad...",12.0,No consta,2025.0
10611,Sant Martí,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",8.0,No consta,2025.0


Also, if we want to know where are the inmigrants with postmandatory studies from and in which districts of Barcelona are they living:

In [59]:
# We read the data
df=pd.read_csv('generated_data/education_continents_1997-2025.csv')

# We delete the 'no consta' because they are negligible
df=df[~df["Titulación académica"].str.strip().str.lower().eq("no consta")]
df=df[~df["Continente"].str.strip().str.lower().eq("no consta")]

# Let's group America and Oceania
df["group_continent"]=df["Continente"].replace({
    "América": "América-Oceanía",
    "Oceanía": "América-Oceanía"
})

# We select the postmandatory levels using keywords
keywords=["Bachillerato", "COU", "FPII", "CFGM", "grado medio", "universitarios", "CFGS"]
mask=df["Titulación académica"].str.contains("|".join(keywords), case=False, na=False)
df_educated=df[mask].copy()

# We save the new dataset
df_educated.to_csv('generated_data/postcompulsory_continents_1997-2025.csv', index=False, encoding='utf-8')
df_educated

,Territorio,Titulación académica,Poblacion,Continente,Año,group_continent
2,Ciutat Vella,"Bachillerato elemental, graduado escolar, ESO,...",129.0,África,1997.0,África
3,Ciutat Vella,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",197.0,África,1997.0,África
4,Ciutat Vella,"Estudios universitarios, CFGS grado superior",73.0,África,1997.0,África
8,Eixample,"Bachillerato elemental, graduado escolar, ESO,...",126.0,África,1997.0,África
9,Eixample,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",280.0,África,1997.0,África
...,...,...,...,...,...,...
7986,Nou Barris,"Estudios universitarios, CFGS grado superior",11.0,Oceanía,2025.0,América-Oceanía
7987,Sant Andreu,"Estudios universitarios, CFGS grado superior",12.0,Oceanía,2025.0,América-Oceanía
7989,Sant Martí,"Bachillerato elemental, graduado escolar, ESO,...",11.0,Oceanía,2025.0,América-Oceanía
7990,Sant Martí,"Bachillerato superior, BUP, COU, FPII, CFGM gr...",17.0,Oceanía,2025.0,América-Oceanía
